In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [1]:
! ln -s /Users/work/_ai/projects/crypto-trade/_crypto-data ./

In [3]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out


In [4]:
# Load and preprocess data
data = pd.read_csv('_crypto-data/BTCUSD_230630-trades-2023-04-01.csv')
data = data[['price', 'qty', 'base_qty']]  # Select relevant columns


In [5]:
# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

sequence_length = 60
X, y = [], []
for i in range(len(data_scaled) - sequence_length):
    X.append(data_scaled[i:i+sequence_length])
    y.append(data_scaled[i+sequence_length, 0])  # We want to predict the price

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1)  # Adding dimension for consistency
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(-1)

# Hyperparameters
input_size = X_train.shape[2]  # Number of features
hidden_size = 50
num_layers = 2
output_size = 1
num_epochs = 100
learning_rate = 0.001

In [6]:
# Model, loss function, and optimizer
model = LSTMModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train)
    optimizer.zero_grad()
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluating the model
model.eval()
with torch.no_grad():
    predicted = model(X_test)
    predicted = scaler.inverse_transform(np.concatenate((predicted.numpy(), np.zeros((predicted.shape[0], 2))), axis=1))[:, 0]
    y_test = scaler.inverse_transform(np.concatenate((y_test.numpy(), np.zeros((y_test.shape[0], 2))), axis=1))[:, 0]
    # Calculate evaluation metrics, such as RMSE
    rmse = np.sqrt(np.mean((predicted - y_test) ** 2))
    print(f'RMSE: {rmse:.4f}')


Epoch [10/100], Loss: 0.1719
Epoch [20/100], Loss: 0.0505
Epoch [30/100], Loss: 0.0339
Epoch [40/100], Loss: 0.0299
Epoch [50/100], Loss: 0.0270
Epoch [60/100], Loss: 0.0222
Epoch [70/100], Loss: 0.0153
Epoch [80/100], Loss: 0.0036
Epoch [90/100], Loss: 0.0025
Epoch [100/100], Loss: 0.0011
RMSE: 22.0226


In [7]:
predict = model._save_to_state_dict()

TypeError: Module._save_to_state_dict() missing 3 required positional arguments: 'destination', 'prefix', and 'keep_vars'